In [43]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [44]:
model = Sequential() #Cria o construtor da rede neural
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu')) #Adiciona uma camada de 32 neuronios 
model.add(MaxPooling2D(pool_size = (2, 2))) #Diminiu o pooling, ou a matriz a ser analisada

model.add(Conv2D(64, (3, 3), activation = 'relu')) #Adiciona uma camada de 64 neuronios
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu')) #Adiciona uma camada de 128 neuronios
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten()) # "Achata a entrada" Multiplica os imput shape e "cria um vetor"

In [45]:
model.add(Dense(units = 64, activation = 'relu')) #Primeira camada de saída
model.add(Dense(units = 30, activation = 'softmax')) #Camada de saída

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) #Cria a rede neural

In [46]:
data_train = ImageDataGenerator(rescale = 1./255,
                                         rotation_range = 7, 
                                         horizontal_flip = True,
                                         shear_range=0.2,
                                         height_shift_range=0.07,
                                         zoom_range=0.2) #Gera as imagens de entrada tratada

data_test = ImageDataGenerator(rescale = 1./255) #Gera as imagens de teste

In [47]:
base_train = data_train.flow_from_directory('simpsons_dataset',
                                            target_size = (64, 64),
                                            batch_size = 30,
                                            class_mode = 'categorical') #Importa conjunto de imagens de treino, com pacotes de 30

base_test = data_test.flow_from_directory('simpsons_testset',
                                          target_size = (64, 64),
                                          batch_size = 30,
                                          class_mode = 'categorical') #Importa conjunto de imagens de teste, com pacotes de 30

Found 20582 images belonging to 30 classes.
Found 3667 images belonging to 30 classes.


In [48]:
model.fit_generator(base_train,
                    steps_per_epoch = 90,
                    epochs = 40,
                    validation_data = base_test,
                    validation_steps = 4) #Coloca a rede pra treinar, com 90 passos por épocas ou
                                          # "90 comparações por época" e com 40 épocas "repetir 40 vezes"

Epoch 1/40
90/90 [==============================] - 28s 307ms/step - loss: 3.0626 - acc: 0.1207 - val_loss: 2.6465 - val_acc: 0.5333
Epoch 2/40
90/90 [==============================] - 28s 312ms/step - loss: 2.7132 - acc: 0.2356 - val_loss: 2.6828 - val_acc: 0.3833
Epoch 3/40
90/90 [==============================] - 28s 310ms/step - loss: 2.3883 - acc: 0.3326 - val_loss: 2.6819 - val_acc: 0.3500
Epoch 4/40
90/90 [==============================] - 28s 309ms/step - loss: 2.2042 - acc: 0.3855 - val_loss: 2.5723 - val_acc: 0.4333
Epoch 5/40
90/90 [==============================] - 28s 309ms/step - loss: 2.0463 - acc: 0.4344 - val_loss: 2.5100 - val_acc: 0.4167
Epoch 6/40
90/90 [==============================] - 28s 313ms/step - loss: 1.8724 - acc: 0.4881 - val_loss: 2.3435 - val_acc: 0.4667
Epoch 7/40
90/90 [==============================] - 29s 325ms/step - loss: 1.8263 - acc: 0.4881 - val_loss: 2.3094 - val_acc: 0.4333
Epoch 8/40
90/90 [==============================] - 29s 318ms/step - 

In [50]:
result = []

test_image = image.load_img('simpsons_testset/abraham_grampa_simpson/pic_0001.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

pred = model.predict_on_batch(test_image)
result.append(pred)

result = np.asarray(result)
imprime = np.array(result[0][0])
print(imprime)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


In [51]:
model_json = model.to_json()

In [52]:
with open('model_json.json', 'w') as json_file:
    json_file.write(model_json)

In [54]:
model.save_weights('model_simpsons.h5')